In [1]:
#Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import time
import math

from numba import jit, cuda, float32, int32, prange
from sklearn.cluster import kmeans_plusplus, KMeans

Save data of processing time of all version (using `pandas`) in order to compare running time.

In [2]:
import pandas as pd
all_results = pd.DataFrame(columns=['kmeans_cal_distance', 'kmeans_assign_labels', 'kmeans_update_centroids', 'kmeans algorithm'], dtype=float)
all_results # Bảng biểu lưu lại thời gian chạy của tất cả các phiên bản

,kmeans_cal_distance,kmeans_assign_labels,kmeans_update_centroids,kmeans algorithm


# Read data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# read data
data=pd.read_csv('/content/drive/My Drive/data_standardize.csv')
df_Standardize=pd.DataFrame(data)
df_Standardize

,Unnamed: 0,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,0,-0.745369,-0.248785,-0.465787,-0.397088,-0.365854,-0.476844,-0.805116,-0.677987,-0.706172,-0.675708,-0.482201,-0.520766,-0.965868,-0.549205,-0.324378,-0.525663,0.360754
1,1,0.822978,0.134866,-0.516362,-0.397088,-0.479339,2.682990,-1.220469,-0.677987,-0.915974,0.576272,0.114210,-0.603824,0.705048,0.877253,0.123457,0.234367,0.360754
2,2,0.472101,0.518517,-0.106471,0.140262,-0.479339,-0.476844,1.271643,2.681612,-0.915974,-0.675708,-0.482201,-0.105476,0.844291,-0.395538,-0.090206,-0.525663,0.360754
3,3,0.061121,-1.016087,0.278323,0.644711,-0.479339,-0.375919,-1.012794,-0.398022,-0.915974,-0.258383,-0.333098,-0.562295,0.844291,-0.622992,-0.391354,-0.525663,0.360754
4,4,-0.360018,0.518517,-0.507880,-0.385968,-0.479339,-0.476844,-1.012794,-0.398022,-0.915974,-0.675708,-0.482201,-0.562295,-0.910171,-0.374964,-0.273834,-0.525663,0.360754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8926,8926,-0.751524,0.518517,-0.362027,-0.397088,-0.133031,-0.476844,1.271643,-0.677987,1.182060,-0.675708,-0.482201,-0.354650,-0.965868,-0.503941,-0.367885,1.184407,-4.124481
8927,8927,-0.756142,0.518517,-0.357319,-0.397088,-0.122468,-0.476844,1.271643,-0.677987,1.182060,-0.675708,-0.482201,-0.354650,-0.965868,-0.522125,-0.258918,-0.525663,-4.124481
8928,8928,-0.754051,-0.184845,-0.439810,-0.397088,-0.307565,-0.476844,0.856290,-0.677987,0.762454,-0.675708,-0.482201,-0.396179,-0.965868,-0.593276,-0.351786,0.329372,-4.124481
8929,8929,-0.758982,-0.184845,-0.516362,-0.397088,-0.479339,-0.458914,-1.220469,-0.677987,-0.915974,0.158947,-0.183995,-0.603824,-1.105111,-0.603778,-0.364587,0.329372,-4.124481


# Define number of clusters

In [5]:
K = 3

# KMeans code from Scikit-learn
Using Scikit-learn to get final cluster to compare with each KMeans function which help to check for the correctness.

In [6]:
init_centers, _ = kmeans_plusplus(df_Standardize.to_numpy(), n_clusters=K, random_state=0)
Kmean_sklearn = KMeans(n_clusters=K, init=init_centers)
Kmean_sklearn.fit(df_Standardize)
correct_centers = Kmean_sklearn.cluster_centers_
correct_centers

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params_vs_input(X, default_n_init=10)


array([[ 4.47600000e+03,  3.78129110e-02,  4.96482720e-02,
         4.17281124e-04,  5.63675708e-03, -8.67095870e-03,
         5.27745180e-03, -4.67940758e-02,  4.52133306e-02,
        -7.04763619e-02,  1.69532137e-02,  1.77968353e-02,
        -5.25913296e-03,  7.60483264e-02,  1.71069245e-02,
         5.06448067e-03, -2.73357568e-02,  7.87632154e-02],
       [ 1.49350000e+03,  2.48677832e-01,  9.48388677e-02,
         1.55379122e-01,  1.50052781e-01,  9.15402064e-02,
         9.38293979e-02,  4.53166002e-02,  1.67794131e-01,
         4.33955530e-03,  1.80479618e-02,  1.39601273e-02,
         1.24963172e-01,  3.69137149e-01,  1.77989294e-01,
         8.75138483e-02, -2.30342762e-02,  1.39344273e-01],
       [ 7.44750000e+03, -2.88475522e-01, -1.45374728e-01,
        -1.56950460e-01, -1.56823444e-01, -8.35160798e-02,
        -9.98223921e-02,  1.31488945e-03, -2.14419740e-01,
         6.63659939e-02, -3.51979188e-02, -3.19265135e-02,
        -1.20611436e-01, -4.48205553e-01, -1.96479880e

# Sequential code version 1: pure KMeans code using Python
In this version, we use pure Python to develop KMeans algorithm. Additionally, because of optimized code we applied `KMeans++` library in order to initial first stage centers which helps reduce loop and stabilize final centers of each cluster as well as processing time

## Function to generate suitable centers for KMeans
Using `Kmeans++` to find optimized centers.
- Input: normalized data, number of clusters
- Output: optimized centers

In [7]:
# initial centers
def kmeans_init_centers(df_Standardize, K):
  centers = []
  init_centers, _ = kmeans_plusplus(df_Standardize.to_numpy(), n_clusters=K, random_state=0)
  for x in init_centers:
    centers.append(x)
  return centers

## Function to calculate distance between each data point
Computes the Euclidean distance between each data point and each cluster center.
- Input: center of each cluster, each data point
- Output: Euclidean distance

In [8]:
# calculate distance between data points using Euclid distance
def kmeans_cal_distance(point_1, point_2):
  # calculate sum of each element (x,y,z,...) of each point
  sum_point = 0
  for i in range(len(point_1)):
    sum_point += (point_1[i] - point_2[i])**2
  return np.sqrt(sum_point)

## Function to assign label for each data point
Assign each data point to the nearest centroid.
- Input: normalized data, centers of each cluster
- Output: list of labeled data points, sum of processing time of function `kmeans_cal_distance`

In [9]:
# assign label for each point corresponding to each cluster that point related
def kmeans_assign_labels(df_Standardize, centers):
  # store label of each data point
  assigned_labels = []

  # store processing time of calculating distance step
  list_time = []
  for i in range(len(df_Standardize)):
    distance = []
    # calculate distance from each data point to each center
    for j in range(len(centers)):
      start = time.time()
      distance.append(kmeans_cal_distance(centers[j], np.array(df_Standardize.iloc[i])))
      end = time.time()
      list_time.append(end-start)
    # find closest center
    index = np.argmin(distance)
    # assign data points to proper cluster
    assigned_labels.append(index)
  return assigned_labels, sum(list_time)

## Function to find new center for each cluster
Calculate the new centroids as the mean of all data points assigned to each cluster.
- Input: normalized data, assigned data points, number of clusters
- Output: new centers correspond to each cluster

In [10]:
# update new centroids
def kmeans_update_centroids(df_Standardize, labels, K):
  centers = []
  # collect all points assigned to the k-th cluster
  # take average
  for k in range(K):
    # store points of each cluster
    cluster_k = []
    for i in range(len(labels)):
      if labels[i] == k:
        cluster_k.append(np.array(df_Standardize.iloc[i]))
    # update new center
    # sum all data point in each cluster
    new_center = np.zeros(df_Standardize.shape[1])
    for i in range(len(cluster_k[0])):
      for j in range(len(cluster_k)):
          new_center[i] += cluster_k[j][i]
    # take average
    for i in range(len(new_center)):
      new_center[i] = new_center[i]/len(cluster_k)
    centers.append(new_center)
  return centers

## Function to find converged stage
Check if new centers and previous centers has no different then stop the algorithm as we get final centers.
- Input: previous centers, new centers
- Output: `True` if satisfy given condition, otherwise return `False`

In [11]:
# stop condition
def has_converged(centers, new_centers):
  # return True if two sets of centers are the same
  for i in range(len(centers)):
    for j in range(len(centers[i])):
        if centers[i][j] != new_centers[i][j]:
            return False
  return True

## Main KMeans function
Iterate through all included function to operate the algorithm:
Input: normalized data, number of clusters
Output: list of all centers, list of all labeled data points, number of iterations

In [12]:
def kmeans(df_Standardize, K):
  # store the center coordinates of each iteration
  centers = [kmeans_init_centers(df_Standardize, K)]
  # save the label of each iteration
  labels = []
  it = 0

  # list store processing time of function calculate distance
  list_time_0 = []
  # list store processing time of function assign labels
  list_time_1 = []
  # list store processing time of function update new centers
  list_time_2 = []

  while True:
    # at each iteration:
    # 1. assign label for each points and append to labels
    # 2. update the centers
    # 3. check the convergence condition and append NEW center coordinates to centers
    # 4. update iteration
    start_1 = time.time()
    assigned_labels, temp = kmeans_assign_labels(df_Standardize, centers[it])
    end_1 = time.time()
    list_time_0.append(temp)
    list_time_1.append(end_1-start_1)

    labels.append(assigned_labels)

    start_2 = time.time()
    new_centers = kmeans_update_centroids(df_Standardize, assigned_labels, K)
    end_2 = time.time()
    list_time_2.append(end_2-start_2)
    if has_converged(centers[it], new_centers):
      break
    else:
      centers.append(new_centers)
      it += 1
  # display processing time of each function
  print(f'Processing time of function kmeans_cal_distance: {sum(list_time_0)/len(list_time_0)} s')
  print(f'Processing time of function kmeans_assign_labels: {sum(list_time_1)/len(list_time_1)} s')
  print(f'Processing time of function kmeans_update_centroids: {sum(list_time_2)/len(list_time_2)} s')
  return centers, labels, it

In [13]:
%%capture output_1
start = time.time()
(centers, labels, it) = kmeans(df_Standardize, K)
end = time.time()
print(f'Processing time: {end - start} s')
print(f'Number of loops: {it}')
print('Centers found by k-means algorithm:')
print(centers[-1])
print('='*60)

## Show result

In [14]:
print(output_1)

Processing time of function kmeans_cal_distance: 2.835097146034241 s
Processing time of function kmeans_assign_labels: 2.98659485578537 s
Processing time of function kmeans_update_centroids: 0.7648953676223755 s
Processing time: 75.04455256462097 s
Number of loops: 19
Centers found by k-means algorithm:
[array([ 4.46700000e+03,  3.81094862e-02,  5.02189895e-02,  5.83274799e-04,
        5.36927127e-03, -7.84065808e-03,  6.31988406e-03, -4.67442461e-02,
        4.47162453e-02, -6.93487645e-02,  1.87956259e-02,  1.88486182e-02,
       -3.75253784e-03,  7.48275532e-02,  1.65785762e-02,  4.62410403e-03,
       -2.75550847e-02,  7.75076913e-02]), array([ 1.48900000e+03,  2.49492112e-01,  9.41476053e-02,  1.55945414e-01,
        1.50572768e-01,  9.19211148e-02,  9.41030059e-02,  4.62326841e-02,
        1.68684573e-01,  4.51414240e-03,  1.73220983e-02,  1.38074106e-02,
        1.24850808e-01,  3.70945501e-01,  1.78682721e-01,  8.81508644e-02,
       -2.33944405e-02,  1.39930043e-01]), array([ 

## Evalute

### Processing time
Report processing time of each function in KMeans algorithm and total processing time of KMeans

In [15]:
all_results.loc['sequential 1: pure Python', 'kmeans_cal_distance'] = output_1.stdout.split()[5]
all_results.loc['sequential 1: pure Python', 'kmeans_assign_labels'] = output_1.stdout.split()[12]
all_results.loc['sequential 1: pure Python', 'kmeans_update_centroids'] = output_1.stdout.split()[19]
all_results.loc['sequential 1: pure Python', 'kmeans algorithm'] = output_1.stdout.split()[23]
all_results.loc['sequential 1: pure Python', 'number of loops'] = it
all_results

<ipython-input-15-e6b5ee322360>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2.835097146034241' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_results.loc['sequential 1: pure Python', 'kmeans_cal_distance'] = output_1.stdout.split()[5]
<ipython-input-15-e6b5ee322360>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2.98659485578537' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_results.loc['sequential 1: pure Python', 'kmeans_assign_labels'] = output_1.stdout.split()[12]
<ipython-input-15-e6b5ee322360>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.7648953676223755' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_results.loc

,kmeans_cal_distance,kmeans_assign_labels,kmeans_update_centroids,kmeans algorithm,number of loops
sequential 1: pure Python,2.835097146034241,2.98659485578537,0.7648953676223755,75.04455256462097,19.0


### Accuracy
Compare the result of this version with the result of KMeans library. If there is no difference then the result is correct.

In [16]:
np.mean(np.abs(correct_centers - centers))

4.369036071459774

# Sequential code version 2: Optimized sequential KMeans code
Using numba with decorator @jit to optimize sequential KMeans code

## Convert data to numpy array in orfer to apply Numba

In [17]:
df_Standardize=df_Standardize.to_numpy()

## Analyze
In this project we realize that there are 3 functions that occupy almost processing time `kmeans_cal_distance`, `kmeans_assign_labels` and `kmeans_update_centroids` so we are going to speed up these functions by running those parelle on CPU.

## Function to generate suitable centers for KMeans
Using `Kmeans++` to find optimized centers.
- Input: normalized data, number of clusters
- Output: optimized centers

In [18]:
def kmeans_init_centers(df_Standardize, K):
  centers, _ = kmeans_plusplus(df_Standardize, n_clusters=K, random_state=0)
  return centers

## Function to calculate distance between each data point
Computes the Euclidean distance between each data point and each cluster center.
- Input: center of each cluster, each data point
- Output: Euclidean distance



In [19]:
@jit(nopython=True)
# calculate distance between data points using Euclid distance
def kmeans_cal_distance(point_1, point_2):
  # calculate sum of each element (x,y,z,...) of each point
  sum_point = 0.0
  for i in range(point_1.shape[0]):
    sum_point += (point_1[i] - point_2[i]) ** 2
  return np.sqrt(sum_point)

## Function to assign label for each data point
Assign each data point to the nearest centroid.
- Input: normalized data, centers of each cluster
- Output: list of labeled data points, sum of processing time of function `kmeans_cal_distance`

In [20]:
@jit(nopython=True)
# assign label for each point corresponding to each cluster that point related
def kmeans_assign_labels(df_Standardize, centers):
  assigned_labels = np.empty(df_Standardize.shape[0], dtype=np.int32)
  # calculate distance from each data point to each center
  for i in range(df_Standardize.shape[0]):
    distance = np.empty(centers.shape[0])
    for j in range(centers.shape[0]):
      distance[j] = kmeans_cal_distance(centers[j, :], df_Standardize[i, :])
    # find closest center and assign data points to proper cluster
    assigned_labels[i] = np.argmin(distance)
  return assigned_labels

## Function to measure processing time of function calculating distance between each data point

Since Numba does not support `time` library so we need to separate the original function that use function calculate ditance to assign label for every data points.

In [21]:
# External function to measure the processing time of kmeans_cal_distance
def measure_distance_time(df_Standardize, centers):
  start = time.time()
  assigned_labels = np.empty(df_Standardize.shape[0], dtype=np.int32)
  for i in range(df_Standardize.shape[0]):
      distance = np.empty(centers.shape[0])
      for j in range(centers.shape[0]):
          distance[j] = kmeans_cal_distance(centers[j, :], df_Standardize[i, :])
      assigned_labels[i] = np.argmin(distance)
  end = time.time()
  return assigned_labels, end - start

## Function to find new center for each cluster
Calculate the new centroids as the mean of all data points assigned to each cluster.
- Input: normalized data, assigned data points, number of clusters
- Output: new centers correspond to each cluster

In [22]:
# update new centroids
@jit(nopython=True)
def kmeans_update_centroids(df_Standardize, labels, K):
  centers = np.zeros((K, df_Standardize.shape[1]))
  counts = np.zeros(K, dtype=np.int32)
  # collect all points assigned to the k-th cluster
  # take average
  for i in range(df_Standardize.shape[0]):
    # update new center
    # sum all data point in each cluster
    label = labels[i]
    centers[label, :] += df_Standardize[i, :]
    counts[label] += 1
  for k in range(K):
    if counts[k] > 0:
      # take average
      centers[k, :] /= counts[k]
  return centers

## Function to find converged stage
Check if new centers and previous centers has no different then stop the algorithm as we get final centers.
- Input: previous centers, new centers
- Output: `True` if satisfy given condition, otherwise return `False`

In [23]:
# stop condition
@jit(nopython=True)
def has_converged(centers, new_centers):
  # return True if two sets of centers are the same
  for i in range(len(centers)):
    for j in range(len(centers[i])):
      if centers[i][j] != new_centers[i][j]:
        return False
  return True

## Main KMeans function
Iterate through all included function to operate the algorithm:
Input: normalized data, number of clusters
Output: list of all centers, list of all labeled data points, number of iterations

In [24]:
# main kmeans function
def kmeans(df_Standardize, K):
  centers = kmeans_init_centers(df_Standardize, K)

  # save the label of each iteration
  labels = []
  it = 0

  # list store processing time of function calculate distance
  list_time_0 = []
  # list store processing time of function assign labels
  list_time_1 = []
  # list store processing time of function update new centers
  list_time_2 = []

  while True:
    # at each iteration:
    # 1. assign label for each points and append to labels
    # 2. update the centers
    # 3. check the convergence condition and append NEW center coordinates to centers
    # 4. update iteration
    start_1 = time.time()
    assigned_labels, temp = measure_distance_time(df_Standardize, centers)
    end_1 = time.time()
    list_time_0.append(temp)
    list_time_1.append(end_1-start_1)

    labels.append(assigned_labels)

    start_2 = time.time()
    new_centers = kmeans_update_centroids(df_Standardize, assigned_labels, K)
    end_2 = time.time()
    list_time_2.append(end_2-start_2)
    if has_converged(centers, new_centers):
        break
    centers = new_centers
    it += 1

  # display processing time of each function
  print(f'Processing time of function kmeans_cal_distance: {sum(list_time_0)/len(list_time_0)} s')
  print(f'Processing time of function kmeans_assign_labels: {sum(list_time_1)/len(list_time_1)} s')
  print(f'Processing time of function kmeans_update_centroids: {sum(list_time_2)/len(list_time_2)} s')
  return centers, assigned_labels, it

In [25]:
%%capture output_2
start = time.time()
(centers, labels, it) = kmeans(df_Standardize, K)
end = time.time()
print(f'Processing time: {end - start} s')
print(f'Number of loops: {it}')
print('Centers found by k-means algorithm:')
print(centers)
print('='*60)

## Show results

In [26]:
print(output_2)

Processing time of function kmeans_cal_distance: 0.08731532096862793 s
Processing time of function kmeans_assign_labels: 0.08731774091720582 s
Processing time of function kmeans_update_centroids: 0.037986922264099124 s
Processing time: 2.6303088665008545 s
Number of loops: 19
Centers found by k-means algorithm:
[[ 4.46700000e+03  3.81094862e-02  5.02189895e-02  5.83274799e-04
   5.36927127e-03 -7.84065808e-03  6.31988406e-03 -4.67442461e-02
   4.47162453e-02 -6.93487645e-02  1.87956259e-02  1.88486182e-02
  -3.75253784e-03  7.48275532e-02  1.65785762e-02  4.62410403e-03
  -2.75550847e-02  7.75076913e-02]
 [ 1.48900000e+03  2.49492112e-01  9.41476053e-02  1.55945414e-01
   1.50572768e-01  9.19211148e-02  9.41030059e-02  4.62326841e-02
   1.68684573e-01  4.51414240e-03  1.73220983e-02  1.38074106e-02
   1.24850808e-01  3.70945501e-01  1.78682721e-01  8.81508644e-02
  -2.33944405e-02  1.39930043e-01]
 [ 7.44300000e+03 -2.87962670e-01 -1.44526941e-01 -1.56738756e-01
  -1.56148099e-01 -8.41

## Evalute

### Processing time
Report processing time of each function in KMeans algorithm and total processing time of KMeans

In [27]:
all_results.loc['sequential 2: optimized KMeans code', 'kmeans_cal_distance'] = output_2.stdout.split()[5]
all_results.loc['sequential 2: optimized KMeans code', 'kmeans_assign_labels'] = output_2.stdout.split()[12]
all_results.loc['sequential 2: optimized KMeans code', 'kmeans_update_centroids'] = output_2.stdout.split()[19]
all_results.loc['sequential 2: optimized KMeans code', 'kmeans algorithm'] = output_2.stdout.split()[23]
all_results.loc['sequential 2: optimized KMeans code', 'number of loops'] = it
all_results

,kmeans_cal_distance,kmeans_assign_labels,kmeans_update_centroids,kmeans algorithm,number of loops
sequential 1: pure Python,2.835097146034241,2.98659485578537,0.7648953676223755,75.04455256462097,19.0
sequential 2: optimized KMeans code,0.08731532096862793,0.08731774091720582,0.037986922264099124,2.6303088665008545,19.0


### Accuracy
Compare the result of this version with the result of KMeans library. If there is no difference then the result is correct.

In [28]:
np.mean(np.abs(correct_centers - centers))

0.3340304859188367

# Sequential code version 3: Paralyzed sequential code running on CPU
Using numba with decorator @jit and prange to paralyze sequential KMeans code

## Analyze
In this project we realize that there are 3 functions that occupy almost processing time `kmeans_cal_distance`, `kmeans_assign_labels` and `kmeans_update_centroids` so we are going to speed up these functions by running those parelle on CPU. In addition, each task in those functions can run independently so we can devide it to each thread in CPU.

## Function to generate suitable centers for KMeans
Using `Kmeans++` to find optimized centers.
- Input: normalized data, number of clusters
- Output: optimized centers

In [29]:
def kmeans_init_centers(df_Standardize, K):
  centers, _ = kmeans_plusplus(df_Standardize, n_clusters=K, random_state=0)
  return centers

## Function to calculate distance between each data point
Computes the Euclidean distance between each data point and each cluster center.
- Input: center of each cluster, each data point
- Output: Euclidean distance



In [30]:
@jit(parallel=True, nopython=True)
# calculate distance between data points using Euclid distance
def kmeans_cal_distance(point_1, point_2):
  # calculate sum of each element (x,y,z,...) of each point
  sum_point = 0.0
  for i in prange(point_1.shape[0]):
    sum_point += (point_1[i] - point_2[i]) ** 2
  return np.sqrt(sum_point)

## Function to assign label for each data point
Assign each data point to the nearest centroid.
- Input: normalized data, centers of each cluster
- Output: list of labeled data points, sum of processing time of function `kmeans_cal_distance`

In [31]:
@jit(parallel=True, nopython=True)
# assign label for each point corresponding to each cluster that point related
def kmeans_assign_labels(df_Standardize, centers):
  assigned_labels = np.empty(df_Standardize.shape[0], dtype=np.int32)

  # store processing time of calculating distance step
  list_time = []
  for i in prange(df_Standardize.shape[0]):
    distance = np.empty(centers.shape[0])
    # calculate distance from each data point to each center
    for j in range(centers.shape[0]):
      start = time.time()
      distance[j] = kmeans_cal_distance(centers[j, :], df_Standardize[i, :])
      end = time.time()
      list_time.append(end-start)
    # find closest center and assign data points to proper cluster
    assigned_labels[i] = np.argmin(distance)
  return assigned_labels, sum(list_time)

## Function to measure processing time of function calculating distance between each data point

Since Numba does not support `time` library so we need to separate the original function that use function calculate ditance to assign label for every data points.

In [32]:
# External function to measure the processing time of kmeans_cal_distance
def measure_distance_time(df_Standardize, centers):
  start = time.time()
  assigned_labels = np.empty(df_Standardize.shape[0], dtype=np.int32)
  for i in range(df_Standardize.shape[0]):
      distance = np.empty(centers.shape[0])
      for j in range(centers.shape[0]):
          distance[j] = kmeans_cal_distance(centers[j, :], df_Standardize[i, :])
      assigned_labels[i] = np.argmin(distance)
  end = time.time()
  return assigned_labels, end - start

## Function to find new center for each cluster
Calculate the new centroids as the mean of all data points assigned to each cluster.
- Input: normalized data, assigned data points, number of clusters
- Output: new centers correspond to each cluster

In [33]:
# update new centroids
@jit(parallel=True, nopython=True)
def kmeans_update_centroids(df_Standardize, labels, K):
  centers = np.zeros((K, df_Standardize.shape[1]))
  counts = np.zeros(K, dtype=np.int32)
  # collect all points assigned to the k-th cluster
  # take average
  for i in prange(df_Standardize.shape[0]):
    # update new center
    # sum all data point in each cluster
    label = labels[i]
    centers[label, :] += df_Standardize[i, :]
    counts[label] += 1
  # take average
  for k in prange(K):
    if counts[k] > 0:
      centers[k, :] /= counts[k]
  return centers

## Function to find converged stage
Check if new centers and previous centers has no different then stop the algorithm as we get final centers.
- Input: previous centers, new centers
- Output: `True` if satisfy given condition, otherwise return `False`

In [34]:
@jit(nopython=True)
# stop condition
def has_converged(centers, new_centers):
  for i in range(centers.shape[0]):
    for j in range(centers.shape[1]):
      if centers[i, j] != new_centers[i, j]:
        return False
  return True

## Main KMeans function
Iterate through all included function to operate the algorithm:
Input: normalized data, number of clusters
Output: list of all centers, list of all labeled data points, number of iterations

In [35]:
# main kmeans function
def kmeans(df_Standardize, K):
  centers = kmeans_init_centers(df_Standardize, K)

  # save the label of each iteration
  labels = []
  it = 0

  # list store processing time of function calculate distance
  list_time_0 = []
  # list store processing time of function assign labels
  list_time_1 = []
  # list store processing time of function update new centers
  list_time_2 = []

  while True:
    # at each iteration:
    # 1. assign label for each points and append to labels
    # 2. update the centers
    # 3. check the convergence condition and append NEW center coordinates to centers
    # 4. update iteration
    start_1 = time.time()
    assigned_labels, temp = measure_distance_time(df_Standardize, centers)
    end_1 = time.time()
    list_time_0.append(temp)
    list_time_1.append(end_1-start_1)

    labels.append(assigned_labels)

    start_2 = time.time()
    new_centers = kmeans_update_centroids(df_Standardize, assigned_labels, K)
    end_2 = time.time()
    list_time_2.append(end_2-start_2)
    if has_converged(centers, new_centers):
        break
    centers = new_centers
    it += 1

  # display processing time of each function
  print(f'Processing time of function kmeans_cal_distance: {sum(list_time_0)/len(list_time_0)} s')
  print(f'Processing time of function kmeans_assign_labels: {sum(list_time_1)/len(list_time_1)} s')
  print(f'Processing time of function kmeans_update_centroids: {sum(list_time_2)/len(list_time_2)} s')
  return centers, assigned_labels, it

In [36]:
%%capture output_3
start = time.time()
(centers, labels, it) = kmeans(df_Standardize, K)
end = time.time()
print(f'Processing time: {end - start} s')
print(f'Number of loops: {it}')
print('Centers found by k-means algorithm:')
print(centers)
print('='*60)

## Show results

In [37]:
print(output_3)

Processing time of function kmeans_cal_distance: 0.3466216683387756 s
Processing time of function kmeans_assign_labels: 0.34662522077560426 s
Processing time of function kmeans_update_centroids: 0.15332248210906982 s
Processing time: 10.103480100631714 s
Number of loops: 19
Centers found by k-means algorithm:
[[ 4.46700000e+03  3.81094862e-02  5.02189895e-02  5.83274799e-04
   5.36927127e-03 -7.84065808e-03  6.31988406e-03 -4.67442461e-02
   4.47162453e-02 -6.93487645e-02  1.87956259e-02  1.88486182e-02
  -3.75253784e-03  7.48275532e-02  1.65785762e-02  4.62410403e-03
  -2.75550847e-02  7.75076913e-02]
 [ 1.48900000e+03  2.49492112e-01  9.41476053e-02  1.55945414e-01
   1.50572768e-01  9.19211148e-02  9.41030059e-02  4.62326841e-02
   1.68684573e-01  4.51414240e-03  1.73220983e-02  1.38074106e-02
   1.24850808e-01  3.70945501e-01  1.78682721e-01  8.81508644e-02
  -2.33944405e-02  1.39930043e-01]
 [ 7.44300000e+03 -2.87962670e-01 -1.44526941e-01 -1.56738756e-01
  -1.56148099e-01 -8.4198

## Evalute

### Processing time
Report processing time of each function in KMeans algorithm and total processing time of KMeans

In [38]:
all_results.loc['sequential 3: paralyzed KMeans code', 'kmeans_cal_distance'] = output_3.stdout.split()[5]
all_results.loc['sequential 3: paralyzed KMeans code', 'kmeans_assign_labels'] = output_3.stdout.split()[12]
all_results.loc['sequential 3: paralyzed KMeans code', 'kmeans_update_centroids'] = output_3.stdout.split()[19]
all_results.loc['sequential 3: paralyzed KMeans code', 'kmeans algorithm'] = output_3.stdout.split()[23]
all_results.loc['sequential 3: paralyzed KMeans code', 'number of loops'] = it

all_results

,kmeans_cal_distance,kmeans_assign_labels,kmeans_update_centroids,kmeans algorithm,number of loops
sequential 1: pure Python,2.835097146034241,2.98659485578537,0.7648953676223755,75.04455256462097,19.0
sequential 2: optimized KMeans code,0.08731532096862793,0.08731774091720582,0.037986922264099124,2.6303088665008545,19.0
sequential 3: paralyzed KMeans code,0.3466216683387756,0.34662522077560426,0.15332248210906982,10.103480100631714,19.0


### Accuracy
Compare the result of this version with the result of KMeans library. If there is no difference then the result is correct.

In [39]:
np.mean(np.abs(correct_centers - centers))

0.3340304859188368

# Paralyzed code: Paralyzed sequential code running on GPU
Using numba and CUDA to paralyze sequential KMeans code and launch it on GPU.

## Read data


In [40]:
data=pd.read_csv('/content/drive/My Drive/data_standardize.csv')
df_Standardize=pd.DataFrame(data)
df_Standardize

,Unnamed: 0,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,0,-0.745369,-0.248785,-0.465787,-0.397088,-0.365854,-0.476844,-0.805116,-0.677987,-0.706172,-0.675708,-0.482201,-0.520766,-0.965868,-0.549205,-0.324378,-0.525663,0.360754
1,1,0.822978,0.134866,-0.516362,-0.397088,-0.479339,2.682990,-1.220469,-0.677987,-0.915974,0.576272,0.114210,-0.603824,0.705048,0.877253,0.123457,0.234367,0.360754
2,2,0.472101,0.518517,-0.106471,0.140262,-0.479339,-0.476844,1.271643,2.681612,-0.915974,-0.675708,-0.482201,-0.105476,0.844291,-0.395538,-0.090206,-0.525663,0.360754
3,3,0.061121,-1.016087,0.278323,0.644711,-0.479339,-0.375919,-1.012794,-0.398022,-0.915974,-0.258383,-0.333098,-0.562295,0.844291,-0.622992,-0.391354,-0.525663,0.360754
4,4,-0.360018,0.518517,-0.507880,-0.385968,-0.479339,-0.476844,-1.012794,-0.398022,-0.915974,-0.675708,-0.482201,-0.562295,-0.910171,-0.374964,-0.273834,-0.525663,0.360754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8926,8926,-0.751524,0.518517,-0.362027,-0.397088,-0.133031,-0.476844,1.271643,-0.677987,1.182060,-0.675708,-0.482201,-0.354650,-0.965868,-0.503941,-0.367885,1.184407,-4.124481
8927,8927,-0.756142,0.518517,-0.357319,-0.397088,-0.122468,-0.476844,1.271643,-0.677987,1.182060,-0.675708,-0.482201,-0.354650,-0.965868,-0.522125,-0.258918,-0.525663,-4.124481
8928,8928,-0.754051,-0.184845,-0.439810,-0.397088,-0.307565,-0.476844,0.856290,-0.677987,0.762454,-0.675708,-0.482201,-0.396179,-0.965868,-0.593276,-0.351786,0.329372,-4.124481
8929,8929,-0.758982,-0.184845,-0.516362,-0.397088,-0.479339,-0.458914,-1.220469,-0.677987,-0.915974,0.158947,-0.183995,-0.603824,-1.105111,-0.603778,-0.364587,0.329372,-4.124481


## Analyze
In this project we realize that there are 3 functions that occupy almost processing time `kmeans_cal_distance`, `kmeans_assign_labels` and `kmeans_update_centroids` so we are going to speed up these functions by running those parelle on CPU. In addition, each task in those functions can run independently so we can devide it to each thread in CPU. Here is a breakdown of our work.

## Function to generate suitable centers for KMeans
Using `Kmeans++` to find optimized centers.
- Input: normalized data, number of clusters
- Output: optimized centers

**Note: This step is not parallelized since it's a simple and infrequent operation**


In [41]:
# Function to initialize centers using KMeans++
def kmeans_init_centers(df_Standardize, K):
  centers, _ = kmeans_plusplus(df_Standardize.to_numpy(), n_clusters=3, random_state=0)
  return np.array(centers)

## Function to calculate distance between each data point
Computes the Euclidean distance between each data point and each cluster center.
- Input: center of each cluster, each data point
- Output: Euclidean distance



Since the Euclidean distance calculation is independent of each data point, parallel execution is a good fit for it. Therefor, we can use each thread to calculate the distance between each data point.
- Threads: each thread is responsible for calculating the distance between a single data point and all cluster centers.
- Blocks:  handles a subset of the data points.

In [42]:
# CUDA function to calculate distances
@cuda.jit
def calculate_distances(data_points, centers, distances):
    idx = cuda.grid(1)
    if idx < data_points.shape[0]:
        for j in range(centers.shape[0]):
            # calculate sum of each element (x,y,z,...) of each point
            sum_point = 0
            for k in range(data_points.shape[1]):
                sum_point += (data_points[idx, k] - centers[j, k]) ** 2
            distances[idx, j] = math.sqrt(sum_point)

## Function to assign label for each data point
Assign each data point to the nearest centroid.
- Input: normalized data, centers of each cluster
- Output: list of labeled data points, sum of processing time of function `kmeans_cal_distance`

Each data point can be independently assigned to the closest cluster. In this step, we iterate over each cluster to determine the minimal distance that each thread can handle effectively.
- Threads: each thread handles the assignment of one data point to a cluster
- Blocks:  handles a subset of the data points

In [43]:
# CUDA function to assign labels
@cuda.jit
def assign_labels(data_points, centers, labels):
    idx = cuda.grid(1)
    if idx < data_points.shape[0]:
        min_dist = float('inf')
        label = -1
        for j in range(centers.shape[0]):
            sum_point = 0
            # calculate distance from each data point to each center
            for k in range(data_points.shape[1]):
                sum_point += (data_points[idx, k] - centers[j, k]) ** 2
            # find closest center and assign data points to proper cluster
            dist = math.sqrt(sum_point)
            if dist < min_dist:
                min_dist = dist
                label = j
        labels[idx] = label

## Function to find new center for each cluster
Calculate the new centroids as the mean of all data points assigned to each cluster.
- Input: normalized data, assigned data points, number of clusters
- Output: new centers correspond to each cluster

Updating centroids involves summing up the data points assigned to each cluster so it's very approprate if each thread sum up each coordinate of corresponding data point. In this step, atomic operations are crucial to avoid race conditions when multiple threads update the same centroid.
- Threads: each thread contributes to updating the sum of the coordinates of data points for a particular cluster
- Blocks:  handles a subset of the data points

In [44]:
# CUDA function to update centroids
@cuda.jit
def update_centroids(data_points, labels, new_centers, counts, K):
    idx = cuda.grid(1)
    # collect all points assigned to the k-th cluster
    # take average
    if idx < data_points.shape[0]:
        label = labels[idx]
        # update new center
        # sum all data point in each cluster
        for k in range(data_points.shape[1]):
            cuda.atomic.add(new_centers, (label, k), data_points[idx, k])
        cuda.atomic.add(counts, label, 1)

## Function to find converged stage
Check if new centers and previous centers has no different then stop the algorithm as we get final centers.
- Input: previous centers, new centers
- Output: `True` if satisfy given condition, otherwise return `False`

In [45]:
# Function to check convergence
def has_converged(centers, new_centers):
  return np.allclose(centers, new_centers)

## Main KMeans function
Iterate through all included function to operate the algorithm:
Input: normalized data, number of clusters
Output: list of all centers, list of all labeled data points, number of iterations

In [46]:
# Main KMeans function
def kmeans(df_Standardize, K):
    data_points = df_Standardize.values
    centers = kmeans_init_centers(df_Standardize, K)

    threads_per_block = 256
    blocks_per_grid = (data_points.shape[0] + threads_per_block - 1) // threads_per_block

    labels = np.zeros(data_points.shape[0], dtype=np.int32)
    distances = np.zeros((data_points.shape[0], K), dtype=np.float32)

    d_data_points = cuda.to_device(data_points)
    d_centers = cuda.to_device(centers)
    d_labels = cuda.to_device(labels)
    d_distances = cuda.to_device(distances)

    # list store processing time of function calculate distance
    list_time_0 = []
    # list store processing time of function assign labels
    list_time_1 = []
    # list store processing time of function update new centers
    list_time_2 = []

    it = 0
    while True:
        # Step 1: Calculate distances
        start_0 = time.time()
        calculate_distances[blocks_per_grid, threads_per_block](d_data_points, d_centers, d_distances)
        cuda.synchronize()
        end_0 = time.time()
        list_time_0.append(end_0-start_0)

        # Step 2: Assign labels
        start_1 = time.time()
        assign_labels[blocks_per_grid, threads_per_block](d_data_points, d_centers, d_labels)
        cuda.synchronize()
        end_1 = time.time()
        list_time_1.append(end_1-start_1)

        # Step 3: Update centroids
        new_centers = np.zeros((K, data_points.shape[1]), dtype=np.float32)
        counts = np.zeros(K, dtype=np.int32)

        d_new_centers = cuda.to_device(new_centers)
        d_counts = cuda.to_device(counts)

        start_2 = time.time()
        update_centroids[blocks_per_grid, threads_per_block](d_data_points, d_labels, d_new_centers, d_counts, K)
        cuda.synchronize()
        end_2 = time.time()
        list_time_2.append(end_2-start_2)

        new_centers = d_new_centers.copy_to_host()
        counts = d_counts.copy_to_host()

        for k in range(K):
            if counts[k] > 0:
                new_centers[k] /= counts[k]

        if has_converged(centers, new_centers):
            break
        else:
            centers = new_centers.copy()
            d_centers = cuda.to_device(centers)
            it += 1

    labels = d_labels.copy_to_host()

    # display processing time of each function
    print(f'Processing time of function kmeans_cal_distance: {sum(list_time_0)/len(list_time_0)} s')
    print(f'Processing time of function kmeans_assign_labels: {sum(list_time_1)/len(list_time_1)} s')
    print(f'Processing time of function kmeans_update_centroids: {sum(list_time_2)/len(list_time_2)} s')
    return centers, labels, it


In [47]:
%%capture output_4
start = time.time()
(centers, labels, it) = kmeans(df_Standardize, K)
end = time.time()
print(f'Processing time: {end - start} s')
print(f'Number of loops: {it}')
print('Centers found by k-means algorithm:')
print(centers)
print('='*60)

## Show results

In [48]:
print(output_4)

Processing time of function kmeans_cal_distance: 0.0006195014227610797 s
Processing time of function kmeans_assign_labels: 0.0003414249579377087 s
Processing time of function kmeans_update_centroids: 0.0005451560219460616 s
Processing time: 3.2951905727386475 s
Number of loops: 1197
Centers found by k-means algorithm:
[[ 4.4660000e+03  3.7760466e-02  5.0219063e-02 -1.5684374e-04
   4.8022033e-03 -8.5307145e-03  6.3325255e-03 -4.6884015e-02
   4.4528421e-02 -6.9841489e-02  1.8935833e-02  1.8898766e-02
  -4.2826943e-03  7.4125685e-02  1.6191840e-02  4.6324437e-03
  -2.7554967e-02  7.7505410e-02]
 [ 1.4885000e+03  2.4947745e-01  9.4007820e-02  1.5588714e-01
   1.5042223e-01  9.2042714e-02  9.4283521e-02  4.5961164e-02
   1.6821919e-01  4.6120300e-03  1.7414359e-02  1.3923972e-02
   1.2470556e-01  3.7125620e-01  1.7886616e-01  8.8154234e-02
  -2.3225363e-02  1.3985266e-01]
 [ 7.4425161e+03 -2.8741819e-01 -1.4430118e-01 -1.5583362e-01
  -1.5533161e-01 -8.3571807e-02 -1.0067799e-01  9.078224

## Evalute

### Processing time
Report processing time of each function in KMeans algorithm and total processing time of KMeans

In [49]:
all_results.loc['paralyzed kmeans on GPU', 'kmeans_cal_distance'] = output_4.stdout.split()[5]
all_results.loc['paralyzed kmeans on GPU', 'kmeans_assign_labels'] = output_4.stdout.split()[12]
all_results.loc['paralyzed kmeans on GPU', 'kmeans_update_centroids'] = output_4.stdout.split()[19]
all_results.loc['paralyzed kmeans on GPU', 'kmeans algorithm'] = output_4.stdout.split()[23]
all_results.loc['paralyzed kmeans on GPU', 'number of loops'] = it
all_results

,kmeans_cal_distance,kmeans_assign_labels,kmeans_update_centroids,kmeans algorithm,number of loops
sequential 1: pure Python,2.835097146034241,2.98659485578537,0.7648953676223755,75.04455256462097,19.0
sequential 2: optimized KMeans code,0.08731532096862793,0.08731774091720582,0.037986922264099124,2.6303088665008545,19.0
sequential 3: paralyzed KMeans code,0.3466216683387756,0.34662522077560426,0.15332248210906982,10.103480100631714,19.0
paralyzed kmeans on GPU,0.0006195014227610797,0.0003414249579377087,0.0005451560219460616,3.2951905727386475,1197.0


### Accuracy
Compare the result of this version with the result of KMeans library. If there is no difference then the result is correct.

In [50]:
np.mean(np.abs(correct_centers - centers))

0.37082767317807713